In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [6]:
def download_data(name_of_file, index, parse_dates, sep, function):
    return pd.read_csv(name_of_file, index_col = index, parse_dates = parse_dates, sep = sep, date_parser = function)
file = os.path.join('.', 'Index', 'IMOEX.csv')
df1 = download_data(file, 'Дата', ['Дата'], ',', imoex)

In [4]:
ticker = lambda x: pd.to_datetime(x, format = '%Y%m%d')
imoex = lambda x: pd.to_datetime(x, format = '%d.%m.%Y')

### 0. Написать функцию собирающую все преобразованные датафреймы, полученные из файлов с расширением .txt в единую таблицу, где названия столбцов будут название файлов без расширения. Вам потребуется считать все файлы с расширением .txt, потом в цикле их загружать по одному, преобразовывать к 1 столбичному формату, и потом столбец присоединять к итоговому датафрейму. Должно получиться что-то такое

###                      AFLT    GAZP   LKOH   ROSN    SBER
### 2008-01-09  94.12  352.69  2079.00  227.74  101.85
### 2008-01-10  95.39  352.40  2062.00  230.60  103.53
### 2008-01-11  95.76  355.91  2036.00  223.38  104.70
### 2008-01-14  96.99  363.70  2004.00  224.00  105.00
### 2008-01-15  98.30  360.24  1990.97  221.98  105.71

In [7]:
file1 = os.path.join('.', 'Ticker', 'GAZP.txt') 
df2 = download_data(file1,'<DATE>', ['<DATE>'], ';', ticker)
def for_ticker(df):
    un = df['<TICKER>'].unique()[0]
    df.rename(columns = {'<CLOSE>':un}, inplace=True)
    df = df.loc[:,[un]]
    return df
for_ticker(df2)    

GAZP
<DATE>            
2007-02-01  287.95
2007-02-02  289.05
2007-02-05  290.12
2007-02-06  290.85
2007-02-07  286.39
...            ...
2019-03-27  149.49
2019-03-28  149.98
2019-03-29  149.61
2019-04-01  154.53
2019-04-02  154.85

[3047 rows x 1 columns]

In [51]:
def all_df():
    df = pd.DataFrame()
    for f in os.listdir(os.path.join('.', 'Ticker')):
        if os.path.isfile(os.path.join('.', 'Ticker', f)):
            df1 = download_data(os.path.join('.', 'Ticker',f),'<DATE>', ['<DATE>'], ';', ticker)
            df1= for_ticker(df1)
            df = df.merge(df1, left_index=True, right_index=True, how='outer')
    return df

print(all_df())    

             AFLT    GAZP     LKOH    ROSN      SBER
<DATE>                                              
2007-02-01  75.74  287.95  2149.09  238.30  92620.00
2007-02-02  74.90  289.05  2154.00  239.82  91760.03
2007-02-05  74.53  290.12  2209.00  239.26  90796.00
2007-02-06  74.33  290.85  2228.88  239.80  91300.01
2007-02-07  71.30  286.39  2224.00  238.69  90900.00
...           ...     ...      ...     ...       ...
2019-03-27  96.10  149.49  5850.00  404.35    214.10
2019-03-28  96.70  149.98  5880.00  407.50    215.07
2019-03-29  96.30  149.61  5894.00  412.00    214.42
2019-04-01  97.78  154.53  5909.00  419.70    217.70
2019-04-02  98.10  154.85  5907.50  421.90    218.92

[3047 rows x 5 columns]


### 1. Собрать единую таблицу с индексом, при этом индексный столбец должен быть первым
### Выкинуть пустые строки
### Сохранить результат в папку result в файл под именем result.csv

In [61]:
df = df1.merge(all_df(), left_index=True, right_index=True, how = 'iner')

             AFLT    GAZP     LKOH    ROSN      SBER
<DATE>                                              
2007-02-01  75.74  287.95  2149.09  238.30  92620.00
2007-02-02  74.90  289.05  2154.00  239.82  91760.03
2007-02-05  74.53  290.12  2209.00  239.26  90796.00
2007-02-06  74.33  290.85  2228.88  239.80  91300.01
2007-02-07  71.30  286.39  2224.00  238.69  90900.00
...           ...     ...      ...     ...       ...
2019-03-27  96.10  149.49  5850.00  404.35    214.10
2019-03-28  96.70  149.98  5880.00  407.50    215.07
2019-03-29  96.30  149.61  5894.00  412.00    214.42
2019-04-01  97.78  154.53  5909.00  419.70    217.70
2019-04-02  98.10  154.85  5907.50  421.90    218.92

[3047 rows x 5 columns]


TypeError: Can only merge Series or DataFrame objects, a <class 'NoneType'> was passed

### 2. Переиндексировать по возрастанию таблицу
### 3. Перевести все данные в формат процентного изменения. (функция pct_change), выкинуть пустые ячейки
### 4. Нарисовать графики дневных данных всех инструментов в одном окне (используйте сначала функцию cumprod - для чего она узнаете в help)

### 5. Вывести относительные данные доходности в год по каждому из столбцов в виде столбичной диаграммы. Для этого вам надо преобразовать данные в таблице используя функцию resample. После этого необходимо построить столбичный график с нанесением сетки, подписей по оси Х в виде номера года, по оси Y значений в процентах, размеров графика (10,6) дюймов

### 6. Нарисовать 20 периодную скользящую среднюю для каждого из тикеров, так же нарисовать линию изменения стоимости самого тикера  (Читаем что такое простая скользящая средняя, и учимся работать со скользящим окном)

### 7. Построить попарные графики индекса и каждого из тикеров. Сделайте выводы в какие акции стоит вкладываться, в какие нет.

### 8. Построить график лучших вложений с разбивкой по годам. Например в 2010 году лучше был газпром, значит рисуем его в 2010 году, потом в 2011 году был лукойл, тогда в 2011 рисуем  Лукойл и т.д. Вам потребуются функции resample, idxmax,max, pivot_table. Итоговый график должен быть в виде столбичной диаграммы